### 回归的一般方法
（1）收集数据：可以使用任意方法。  
（2）准备数据：回归需要数值型数据，标称型数据将被转成二值型数据。  
（3）分析数据：绘出数据的可视化二维图将有助于对数据做出理解和分析，在采用缩减法求得新回归系数之后，可以将新拟合线绘在图上做对比。  
（4）训练算法：找到回归系数。  
（5）测试算法：使用R2或者预测值和数据的拟合度，来分析模型的效果。  
（6）使用算法：使用回归，可以在给定输入的时候预测出一个数值，这是对分类方法的提升，因为这样可以预测连续型数据而不仅仅是离散的类别标签。

In [1]:
import numpy as np

"""
函数说明:自适应数据加载函数

Parameters:
    fileName - 文件名
Returns:
    dataMat- 数据集
    labelMat- 标签列表
"""
def loadDataSet(fileName):
    numFeat = len(open(fileName).readline().split('\t'))    #获取特征数目
    dataMat = []; labelMat = []
    fr = open(fileName) #打开文件
    for line in fr.readlines(): #逐行读取
        lineArr = []
        curLine = line.strip().split('\t')  #去掉回车，放入列表
        for i in range(numFeat-1):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr) #添加数据
        labelMat.append(float(curLine[-1])) #添加标签
    return dataMat,labelMat

In [2]:
"""
函数说明:计算最佳拟合曲线

Parameters:
    xArr - x数据集
    yArr - y数据集
Returns:
    ws - 回归系数
"""
def standRegres(xArr, yArr):
    xMat = np.mat(xArr); yMat = np.mat(yArr).T  #将x和y数据集转换为numpy矩阵
    xTx = xMat.T*xMat   #根据公式求回归系数
    if np.linalg.det(xTx) == 0.0:   #若矩阵的行列式等于0，则矩阵不可逆
        print("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I*(xMat.T*yMat)    #根据公式求得回归系数
    return ws

In [3]:
"""
函数说明:使用局部加权回归方法计算回归系数

Parameters:
    testPoint - 测试样本点
    xArr - x数据集
    yArr - y数据集
    k - 高斯核的k，自定义系数
Returns:
    testPoint*ws - 回归系数
"""
def lwlr(testPoint, xArr, yArr, k=1.0):
    xMat = np.mat(xArr); yMat = np.mat(yArr)    #读入数据并创建矩阵
    m = np.shape(xMat)[0]       #获得样本个数
    weights = np.mat(np.eye((m)))   #创建对角矩阵
    for j in range(m):      #权重值大小以指数级衰减
        diffMat = testPoint - xMat[j,:]
        weights[j,j] = np.exp(diffMat*diffMat.T/(-2.0*k**2))
    xTx = xMat.T*(weights*xMat)     #根据公式计算回归系数
    if np.linalg.det(xTx)==0.0:
        print("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I*(xMat.T*(weights*yMat.T))
    return testPoint*ws

In [4]:
"""
函数说明:局部线性加权回归测试

Parameters:
    testArr - 测试数据集
    xArr - x数据集
    yArr - y数据集
    k - 高斯核的k，自定义系数
Returns:
    testPoint*ws - 回归系数
"""
def lwlrTest(testArr,xArr,yArr,k=1.0):
    m = np.shape(testArr)[0]    #返回测试数据集的大小
    yHat = np.zeros(m)
    for i in range(m):      #对每个样本点进行预测
        yHat[i] = lwlr(testArr[i],xArr,yArr,k)
    return yHat

In [5]:
"""
函数说明:计算平方误差

Parameters:
    yArr - 实际值
    yHatArr - 预测值
Returns:
    平方误差
"""
def rssError(yArr, yHatArr):
    return ((yArr-yHatArr)**2).sum()

In [6]:
#预测鲍鱼的年龄
abX,abY = loadDataSet("data\\abalone.txt")
yHat01 = lwlrTest(abX[0:99],abX[0:99],abY[0:99],0.1)
yHat1 = lwlrTest(abX[0:99], abX[0:99], abY[0:99], 1)
yHat10 = lwlrTest(abX[0:99], abX[0:99], abY[0:99], 10)

In [7]:
rssError(abY[0:99],yHat01.T)

56.788687430500921

In [8]:
rssError(abY[0:99],yHat1.T)

429.89056187038

In [9]:
rssError(abY[0:99],yHat10.T)

549.11817088279236

由上述结果可知，使用较小的核可以得到较低的误差。但是使用较小的核将造成过拟合。

In [13]:
yHat01 = lwlrTest(abX[100:199],abX[0:99],abY[0:99],0.1)
rssError(abY[100:199],yHat01.T)

57913.515501559108

In [12]:
yHat1 = lwlrTest(abX[100:199],abX[0:99],abY[0:99],1)
rssError(abY[100:199],yHat1.T)

573.5261441895982

In [14]:
yHat10 = lwlrTest(abX[100:199],abX[0:99],abY[0:99],10)
rssError(abY[100:199],yHat10.T)

517.57119053819031

核大小为10时测试误差最小，但是在训练集上的误差却是最大的。

In [15]:
ws = standRegres(abX[0:99],abY[0:99])
yHat = np.mat(abX[100:199])*ws

In [16]:
rssError(abY[100:199],yHat.T.A)

518.63631532455418

简单线性回归达到了与局部加权线性回归类似的效果。表明必须在未知数据上比较效果才能选取最佳模型。